In [2]:
import sys
sys.path.append('../..')
import pandas as pd
import src.cleaning as clean
import seaborn as sns
import src.analysis.utils as utils
import numpy as np
import scipy.stats as stats

In [3]:
sales_data = pd.read_csv('../../data/anon_data.csv')
cleaned_data = clean.clean_data(sales_data)

C:\Users\C\AppData\Local\Temp\ipykernel_17972\1800904834.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_data = pd.read_csv('../../data/anon_data.csv')


In [4]:
# Keep in refunds as refunded/returned items shouldn't have been ordered.

In [5]:
cleaned_data['product_type'].unique().shape

(423,)

In [68]:
product_type_sold_count = cleaned_data.groupby('product_type')['quantity'].sum().sort_values(ascending=False)
# top_sold_product_types = product_type_sold_count[product_type_sold_count > product_type_sold_count.quantile(0.9)]
top_sold_product_types = product_type_sold_count.iloc[list(range(20))]
top_sold_product_types

product_type
Monitors                       350181
Headset                        271576
Notebook Computers             169476
Memory Cards                   134475
Notebook Cases                 107997
Internal SSD                   106580
Motherboards                    96921
Computing Docks & Port Reps     84213
Keyboards                       83899
Mouse/Trackballs                83573
Graphics Cards                  78257
Keyboard and Mouse              76998
Notebook Warranty               73169
Internal HDD                    72124
Professional Notebook           66055
Pen Or Flash Drive              62639
Inkjet Printers                 59828
Home Notebook                   59641
Ink Cartridges                  56131
Professional Desktop PC         47650
Name: quantity, dtype: int32

In [69]:
top_selling_product_filtered_data = cleaned_data.loc[cleaned_data['product_type'].isin(top_sold_product_types.index), :]
top_selling_product_filtered_data

,customer_id,sales_order_number,sales_order_line_number,ship_date,quantity,brand,product_type,product_supplier_id,revenue,profit,freight_cost,sales_team_name
3,100001590,2470475,10,2021-04-02,10,Lenovo,Notebook Computers,1000133.0,21362.979644,133.944138,0.0,Corporate South
4,100001590,2544987,10,2021-04-02,5,Lenovo,Notebook Computers,1000133.0,4344.370602,27.639497,0.0,Corporate South
5,100001590,2601213,30,2021-04-02,3,Lenovo,Notebook Computers,1000133.0,5860.142208,36.556402,0.0,Corporate South
6,100001590,2670696,40,2021-04-02,2,Lenovo,Notebook Computers,1000133.0,725.877833,4.926094,0.0,Corporate South
7,100001590,2670712,40,2021-04-02,2,Lenovo,Notebook Computers,1000133.0,714.169759,5.154048,0.0,Corporate South
...,...,...,...,...,...,...,...,...,...,...,...,...
488698,100008245,60583936,10,2023-03-10,-1,Dell,Monitors,1000003.0,-142.518084,-2.247894,0.0,Corporate South
488699,100008666,60584579,10,2023-03-10,-1,Dell,Home Notebook,1000003.0,-480.761935,-8.915972,0.0,SMB 1 South
488705,100009532,60584483,10,2023-03-10,-1,Poly,Headset,1000037.0,-23.027610,-0.897937,0.0,Corporate South
488708,100014212,60584292,10,2023-03-10,-1,Epson,Ink Cartridges,1000056.0,-136.971448,-11.899225,0.0,Office Automation


In [74]:
pivotted_product = top_selling_product_filtered_data.pivot_table(values='quantity', index=['customer_id'], columns=['product_type'])
correlation_product = pivotted_product.fillna(0).corr().round(2)
# Could probably write some code to sort this to get highest correlations on the top left slowly decreasing to bottom left but sorting sub dataframes in iterations and appending back together
html = correlation_product.style.format(precision=2).background_gradient(cmap='flare').text_gradient(cmap='flare')
html.to_html('../../images/item_correlation/item_correlation_table.html')


In [76]:
correlation_product.style.format(precision=2).background_gradient(cmap='flare')

product_type,Computing Docks & Port Reps,Graphics Cards,Headset,Home Notebook,Ink Cartridges,Inkjet Printers,Internal HDD,Internal SSD,Keyboard and Mouse,Keyboards,Memory Cards,Monitors,Motherboards,Mouse/Trackballs,Notebook Cases,Notebook Computers,Notebook Warranty,Pen Or Flash Drive,Professional Desktop PC,Professional Notebook
product_type,,,,,,,,,,,,,,,,,,,,
Computing Docks & Port Reps,1.00,0.04,0.01,0.02,0.01,0.12,0.01,0.01,0.04,0.02,-0.00,0.18,-0.00,-0.10,0.01,0.16,0.09,0.01,0.09,0.04
Graphics Cards,0.04,1.00,0.01,0.01,0.03,0.01,0.30,0.23,0.05,0.02,0.00,0.04,0.39,0.10,0.09,0.03,0.03,0.03,0.06,0.05
Headset,0.01,0.01,1.00,-0.00,-0.01,-0.00,0.00,-0.00,0.00,0.01,-0.00,0.01,-0.00,0.01,0.28,0.00,0.00,0.00,0.00,-0.00
Home Notebook,0.02,0.01,-0.00,1.00,-0.00,0.04,-0.00,0.00,0.01,-0.00,0.01,0.01,-0.00,0.00,0.00,0.05,0.01,-0.00,0.01,0.69
Ink Cartridges,0.01,0.03,-0.01,-0.00,1.00,0.09,0.01,-0.00,-0.00,-0.00,0.01,-0.02,-0.00,0.00,0.00,-0.01,0.01,0.01,0.02,-0.01
Inkjet Printers,0.12,0.01,-0.00,0.04,0.09,1.00,0.00,-0.00,0.02,0.01,0.01,0.01,0.00,0.16,0.01,0.12,0.01,0.00,-0.00,0.09
Internal HDD,0.01,0.30,0.00,-0.00,0.01,0.00,1.00,0.24,0.10,0.02,0.01,0.05,0.51,0.08,0.01,-0.00,0.00,0.01,0.01,-0.00
Internal SSD,0.01,0.23,-0.00,0.00,-0.00,-0.00,0.24,1.00,0.05,0.01,0.03,0.02,0.26,0.04,0.02,-0.00,0.00,0.01,0.01,0.00
Keyboard and Mouse,0.04,0.05,0.00,0.01,-0.00,0.02,0.10,0.05,1.00,0.27,0.02,0.04,0.07,0.16,0.01,0.02,0.01,0.01,0.02,0.01


In [19]:
bran_product_type_sold_count = cleaned_data.groupby(['brand', 'product_type'])['quantity'].sum().sort_values(ascending=False)
top_sold_brand_product_types = bran_product_type_sold_count[bran_product_type_sold_count > bran_product_type_sold_count.quantile(0.9)]
top_sold_brand_product_types.sum()

3082658

In [22]:
cleaned_data['quantity'].sum() - top_sold_brand_product_types.sum()

302153

In [24]:
brand_product_index_set = cleaned_data.set_index(['brand', 'product_type'])
top_selling_brand_product_filtered_data = brand_product_index_set.loc[top_sold_brand_product_types.index, :].reset_index(level=[0, 1])

In [44]:
pivotted_brand_product = top_selling_brand_product_filtered_data.pivot_table(values='quantity', index=['customer_id'], columns=['brand', 'product_type'])
pivotted_brand_product.fillna(0).corr().sort_values(by=('AOC', 'Gaming Monitor'), ascending=False)

brand                                        AOC                             
product_type                     Gaming Monitor       Monitors               
brand     product_type                                                       
AOC       Gaming Monitor                     1.000000             0.046717  \
Acer      Monitors                           0.442655             0.104286   
Microsoft Keyboards                          0.221180             0.015135   
Evga      Power Supply Units                 0.185749             0.084531   
Dynabook  Notebook Warranty                  0.175400             0.022587   
...                                               ...                  ...   
Norton    Network Routers                   -0.001072            -0.002380   
Flexson   Audio Stands & Mounts             -0.001106            -0.002461   
Speck     Mobile Phone Cases                -0.001326            -0.004147   
LG        Television Accessories            -0.001559             0.041295   
Dynabook  Notebook Computers                -0.003230             0.008958   

brand                                        APC                             
product_type                     Line Interactive UPS UPS Batteries          
brand     product_type                                                       
AOC       Gaming Monitor                     0.007166             0.015283  \
Acer      Monitors                           0.050810             0.048309   
Microsoft Keyboards                          0.023516             0.010977   
Evga      Power Supply Units                -0.002154            -0.002604   
Dynabook  Notebook Warranty                  0.027920             0.022313   
...                                               ...                  ...   
Norton    Network Routers                   -0.002475            -0.003329   
Flexson   Audio Stands & Mounts             -0.004287            -0.003436   
Speck     Mobile Phone Cases                 0.025806            -0.005741   
LG        Television Accessories            -0.004717            -0.003516   
Dynabook  Notebook Computers                -0.001588            -0.000733   

brand                                        Acer                            
product_type                     2 In 1 Notebook      Chromebook             
brand     product_type                                                       
AOC       Gaming Monitor                     0.007877            -0.000053  \
Acer      Monitors                           0.057304             0.023160   
Microsoft Keyboards                          0.003845             0.000565   
Evga      Power Supply Units                 0.004232            -0.000535   
Dynabook  Notebook Warranty                  0.056711            -0.021317   
...                                               ...                  ...   
Norton    Network Routers                   -0.000978             0.159755   
Flexson   Audio Stands & Mounts             -0.000876            -0.001136   
Speck     Mobile Phone Cases                -0.002074             0.002203   
LG        Television Accessories            -0.001474            -0.001544   
Dynabook  Notebook Computers                 0.001472             0.021818   

brand                                                                        
product_type                     Home Notebook        Monitors               
brand     product_type                                                       
AOC       Gaming Monitor                     0.003046             0.442655  \
Acer      Monitors                           0.024907             1.000000   
Microsoft Keyboards                          0.004208             0.035951   
Evga      Power Supply Units                -0.000036             0.008718   
Dynabook  Notebook Warranty                  0.017763             0.081923   
...                                               ...                  ...   
Norton    Network R

In [42]:
pivotted_brand_product.fillna(0).corr()[('AOC', 'Gaming Monitor')]

brand  product_type        
AOC    Gaming Monitor          1.000000
       Monitors                0.046717
APC    Line Interactive UPS    0.007166
       UPS Batteries           0.015283
Acer   2 In 1 Notebook         0.007877
                                 ...   
WD     Internal HDD            0.074274
       Internal SSD            0.063548
       Portable HDD            0.062673
Wacom  Graphics Tablet         0.075556
Yale   Security Devices       -0.001027
Name: (AOC, Gaming Monitor), Length: 222, dtype: float64